In [1]:
import micromagneticmodel as mm
import oommfc as mc

import discretisedfield as df  # df is here chosen to be an alias for discretisedfield

p1 = (0, 0, 0)
p2 = (100e-9, 50e-9, 20e-9)

region = df.Region(p1=p1, p2=p2)

In [2]:
import pyvista as pv

pv.set_jupyter_backend("trame")

In [3]:
region.pyvista(filename="test.svg")

Widget(value="<iframe src='http://localhost:39109/index.html?ui=P_0x7f9a92289d60_0&reconnect=auto' style='widt…

In [4]:
lx, ly, lz = 100e-9, 50e-9, 20e-9

subregions = {
    "bottom_subregion": df.Region(p1=(20e-9, 0, 0), p2=(40e-9, 50e-9, 10e-9)),
    "top_subregion": df.Region(p1=(80e-9, 40e-9, lz / 2), p2=(lx, ly, lz)),
}

cell = (5e-9, 5e-9, 5e-9)

region = df.Region(p1=(0, 0, 0), p2=(lx, ly, lz))
mesh = df.Mesh(region=region, cell=cell, subregions=subregions)

In [19]:
plotter = pv.Plotter()
mesh.pyvista(plotter=plotter, wireframe=True)
plotter.show()

Widget(value="<iframe src='http://localhost:39109/index.html?ui=P_0x7f9a202137c0_13&reconnect=auto' style='wid…

ERROR:root:Exception raised
ERROR:root:KeyError('451d6db20c7d300b91a6f9f15c339d02_80d')
ERROR:root:Traceback (most recent call last):
  File "/home/holtsamu/anaconda3/envs/ubermagdev/lib/python3.8/site-packages/wslink/protocol.py", line 321, in onMessage
    results = func(*args, **kwargs)
  File "/home/holtsamu/anaconda3/envs/ubermagdev/lib/python3.8/site-packages/trame_vtk/modules/vtk/protocols/local_rendering.py", line 33, in get_array
    self.context.get_cached_data_array(data_hash, binary)
  File "/home/holtsamu/anaconda3/envs/ubermagdev/lib/python3.8/site-packages/trame_vtk/modules/vtk/serializers/synchronization_context.py", line 30, in get_cached_data_array
    cache_obj = self.data_array_cache[p_md5]
KeyError: '451d6db20c7d300b91a6f9f15c339d02_80d'



In [6]:
a, b, c = 5e-9, 3e-9, 2e-9
cell = (0.5e-9, 0.5e-9, 0.5e-9)

mesh = df.Mesh(p1=(-a, -b, -c), p2=(a, b, c), cell=cell)


def norm_fun(pos):
    x, y, z = pos
    if (x / a) ** 2 + (y / b) ** 2 + (z / c) ** 2 <= 1:
        return 1e6
    else:
        return 0


def value_fun(pos):
    x, y, z = pos
    c = 1e9
    return (-c * y, c * x, c * z)


field = df.Field(mesh, nvdim=3, value=value_fun, norm=norm_fun, valid="norm")

In [7]:
field.vdims

['x', 'y', 'z']

In [8]:
field.orientation.pyvista.scalar()

Widget(value="<iframe src='http://localhost:39109/index.html?ui=P_0x7f9a92256880_2&reconnect=auto' style='widt…

In [9]:
field.pyvista.vector(cmap="viridis", scalars="x")

Widget(value="<iframe src='http://localhost:39109/index.html?ui=P_0x7f9a75acb580_3&reconnect=auto' style='widt…

In [10]:
field.pyvista.vector(cmap="viridis", scalars="x")

Widget(value="<iframe src='http://localhost:39109/index.html?ui=P_0x7f9a8f8599d0_4&reconnect=auto' style='widt…

In [11]:
field.x.pyvista.scalar(opacity=[1, 0, 1])

Widget(value="<iframe src='http://localhost:39109/index.html?ui=P_0x7f9a75ad1df0_5&reconnect=auto' style='widt…

In [12]:
field.pyvista.contour(isosurfaces=5)

Widget(value="<iframe src='http://localhost:39109/index.html?ui=P_0x7f9a75ad1160_6&reconnect=auto' style='widt…

In [15]:
field.pyvista.contour(isosurfaces=10, scalars="y")

Widget(value="<iframe src='http://localhost:39109/index.html?ui=P_0x7f9a75a65880_9&reconnect=auto' style='widt…

In [16]:
field.pyvista.streamlines(scalars="y")

Widget(value="<iframe src='http://localhost:39109/index.html?ui=P_0x7f9a52eb6b80_10&reconnect=auto' style='wid…

In [17]:
field.pyvista.valid(opacity=0.5, culling="backface")

Widget(value="<iframe src='http://localhost:39109/index.html?ui=P_0x7f9a8fc92e80_11&reconnect=auto' style='wid…

In [18]:
plotter = pv.Plotter()
field.pyvista.vector(plotter=plotter, cmap="viridis")
field.pyvista.contour(plotter=plotter, isosurfaces=10, scalars="x")
plotter.show()

Widget(value="<iframe src='http://localhost:39109/index.html?ui=P_0x7f9a75a6b340_12&reconnect=auto' style='wid…